In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
!pip install pykrx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# from pykrx import stock 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# 구글 마운트

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/NLP_PJT_3

/content/drive/MyDrive/NLP_PJT_3


In [5]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 KB 29.0 MB/s eta 0:00:00


In [6]:
import soynlp
from soynlp.noun import LRNounExtractor_v2
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

import warnings

warnings.filterwarnings('ignore')

# 1. 데이터 로드 및 확인
- 라벨링해서 나눈 train으로 다시 돌려보고, 수작업 및 감성사전 라벨링 성능 비교해보기

In [21]:
target = pd.read_excel('./handlabeling.xlsx').drop("Unnamed: 0", axis=1)

In [22]:
target.head()

,날짜,긍정1 / 부정2 / 쓸데없는거3?,제목+내용,기업명,조회수,공감수,비공감수,페이지,전날_대비_상승_하락,정제된 내용
0,2023-02-09,3.0,그렇다면 ㅣ ㅎ ㅏ나님 야훼가 요구하는 믿음은 무엇인가 믿음으로 모세는 바로의 아...,삼성전자,419.0,0.0,0.0,197.0,-1.0,그렇다면 나님 야훼가 요구하는 믿음은 무엇인가 믿음으로 모세는 바로의 아들이라 칭함...
1,2023-02-09,1.0,나 콜옵 6장 넣었다규 ㆍㆍㆍ? ㆍㆍ감사,삼성전자,220.0,0.0,0.0,197.0,-1.0,나 콜옵 장 넣었다규 감사
2,2023-02-09,2.0,중국풍선 40개국이상 스파이짓속보 삼성게시판은 잡벌레집합소됫노ㅋ,삼성전자,212.0,2.0,0.0,197.0,-1.0,중국풍선 개국이상 스파이짓속보 삼성게시판은 잡벌레집합소됫노
3,2023-02-09,3.0,조나단 에드워드에 의하면 거의 다ㅡ 지옥간다. 신앙은 철처히 양심과 그리스도로 인...,삼성전자,220.0,0.0,2.0,197.0,-1.0,조나단 에드워드에 의하면 거의 다 지옥간다 신앙은 철처히 양심과 그리스도로 인한 신...
4,2023-02-09,3.0,★ 낭비없는 시정운영? 오세훈의 용산 따라하기 https://youtu.be/Dk...,삼성전자,228.0,1.0,0.0,197.0,-1.0,낭비없는 시정운영 오세훈의 용산 따라하기


In [42]:
target.columns

Index(['날짜', '긍정1 / 부정2 / 쓸데없는거3?', '제목+내용', '기업명', '조회수', '공감수', '비공감수',
       '페이지', '전날_대비_상승_하락', '정제된 내용'],
      dtype='object')

In [44]:
target.rename(columns={'긍정1 / 부정2 / 쓸데없는거3?' : 'label'}, inplace=True)
target

,날짜,label,제목+내용,기업명,조회수,공감수,비공감수,페이지,전날_대비_상승_하락,정제된 내용
0,2023-02-09,3.0,그렇다면 ㅣ ㅎ ㅏ나님 야훼가 요구하는 믿음은 무엇인가 믿음으로 모세는 바로의 아...,삼성전자,419.0,0.0,0.0,197.0,-1.0,그렇다면 나님 야훼가 요구하는 믿음은 무엇인가 믿음으로 모세는 바로의 아들이라 칭함...
1,2023-02-09,1.0,나 콜옵 6장 넣었다규 ㆍㆍㆍ? ㆍㆍ감사,삼성전자,220.0,0.0,0.0,197.0,-1.0,나 콜옵 장 넣었다규 감사
2,2023-02-09,2.0,중국풍선 40개국이상 스파이짓속보 삼성게시판은 잡벌레집합소됫노ㅋ,삼성전자,212.0,2.0,0.0,197.0,-1.0,중국풍선 개국이상 스파이짓속보 삼성게시판은 잡벌레집합소됫노
3,2023-02-09,3.0,조나단 에드워드에 의하면 거의 다ㅡ 지옥간다. 신앙은 철처히 양심과 그리스도로 인...,삼성전자,220.0,0.0,2.0,197.0,-1.0,조나단 에드워드에 의하면 거의 다 지옥간다 신앙은 철처히 양심과 그리스도로 인한 신...
4,2023-02-09,3.0,★ 낭비없는 시정운영? 오세훈의 용산 따라하기 https://youtu.be/Dk...,삼성전자,228.0,1.0,0.0,197.0,-1.0,낭비없는 시정운영 오세훈의 용산 따라하기
...,...,...,...,...,...,...,...,...,...,...
20527,2023-02-01,1.0,포케 매수 추천 했던분 작년부터 포케 매수 하라고했던분 그대가 진정 갓 입니다,SK아이이테크놀로지,333.0,2.0,0.0,36.0,1.0,포케 매수 추천 했던분 작년부터 포케 매수 하라고했던분 그대가 진정 갓 입니다
20528,2023-02-01,1.0,고민되네 팔까말까20퍼면 많이 먹긴했는데,SK아이이테크놀로지,409.0,2.0,1.0,36.0,1.0,고민되네 팔까말까 퍼면 많이 먹긴했는데
20529,2023-02-01,2.0,결국 파란색이네7만 왔을때 못팔았다다시 본전오면 팔께,SK아이이테크놀로지,410.0,1.0,1.0,36.0,1.0,결국 파란색이네 만 왔을때 못팔았다다시 본전오면 팔께
20530,2023-02-01,1.0,곧 미국 투자발표나면 보자 쉽알공매들아 미국 분리막 독점이다,SK아이이테크놀로지,816.0,8.0,1.0,36.0,1.0,곧 미국 투자발표나면 보자 쉽알공매들아 미국 분리막 독점이다


## train, test 분리
- 감성분석을 위해 train, test 데이터로 분할
- train 데이터에만 따로 라벨링 후, koBERT 통해 test 데이터 label 예측 

In [67]:
d_train = target.sample(frac=0.70, random_state=2023)
d_test = target.drop(d_train.index)

print(d_train.shape, d_test.shape)

(14372, 10) (6160, 10)


In [68]:
d_train

,날짜,label,제목+내용,기업명,조회수,공감수,비공감수,페이지,전날_대비_상승_하락,정제된 내용
1768,2023-02-06,3.0,매일 지각하는 왕 이 다스리는 나라...대한민국매일 취해사는 왕이 다스리는 나라,삼성전자,272.0,6.0,2.0,432.0,-1.0,매일 지각하는 왕 이 다스리는 나라 대한민국매일 취해사는 왕이 다스리는 나라
13847,2023-02-08,2.0,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 그래도 아직 귀엽다고 본다~~~ 망...,SKC,272.0,0.0,0.0,5.0,-1.0,그래도 아직 귀엽다고 본다 망씨랑 헬쥐중국건강 못따라가
20079,2023-02-08,2.0,공매도 진짜 대단하네ㅋㅋㅋㅋㅋㅋㅋㅋㅋ ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,SK아이이테크놀로지,577.0,1.0,0.0,11.0,-1.0,공매도 진짜 대단하네
14889,2023-02-03,2.0,띨엠전자우 자나깨나 암 조심,삼성전자우,373.0,5.0,12.0,34.0,1.0,띨엠전자우 자나깨나 암 조심
12934,2023-02-02,2.0,휴 시외 5490에샀다가 물타서 겨우탈출했네 평단5310 5340매도... ...,코오롱모빌리티그룹,203.0,2.0,0.0,86.0,-1.0,휴 시외 에샀다가 물타서 겨우탈출했네 평단 매도 원에 추매 해서 관망좀할게요 무섭네...
...,...,...,...,...,...,...,...,...,...,...
13916,2023-02-07,1.0,악재는 소진되었고 이제부터 쭈욱 올라간다 지금잡아라 ~ 1차 115000원 물...,SKC,509.0,2.0,2.0,9.0,1.0,악재는 소진되었고 이제부터 쭈욱 올라간다 지금잡아라 차 원 물들어 올때 노저어라
1291,2023-02-07,3.0,고수로 인증받으려면? 최소 100억은 벌어야지??? ㅋㅋㅋ이 이후로부터는 돈이 돈...,삼성전자,265.0,0.0,0.0,370.0,1.0,고수로 인증받으려면 최소 억은 벌어야지 이 이후로부터는 돈이 돈을 벌게 하므로 그냥...
13257,2023-02-08,1.0,ai 비서?… 이랏샤이마세 여깁니다 형님들,웹케시,362.0,2.0,1.0,2.0,1.0,비서 이랏샤이마세 여깁니다 형님들
15978,2023-02-06,3.0,커피니? 맥심엄? 맥시멈을 맥심엄으로쓰는건 첨본다 푸하하하하하kida가 넘본다 했...,위메이드,373.0,0.0,0.0,81.0,1.0,커피니 맥심엄 맥시멈을 맥심엄으로쓰는건 첨본다 푸하하하하하 가 넘본다 했더니쫄았나 ...


In [69]:
d_train.reset_index(inplace=True)
d_train.drop('index',axis=1,inplace=True)
d_train

,날짜,label,제목+내용,기업명,조회수,공감수,비공감수,페이지,전날_대비_상승_하락,정제된 내용
0,2023-02-06,3.0,매일 지각하는 왕 이 다스리는 나라...대한민국매일 취해사는 왕이 다스리는 나라,삼성전자,272.0,6.0,2.0,432.0,-1.0,매일 지각하는 왕 이 다스리는 나라 대한민국매일 취해사는 왕이 다스리는 나라
1,2023-02-08,2.0,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 그래도 아직 귀엽다고 본다~~~ 망...,SKC,272.0,0.0,0.0,5.0,-1.0,그래도 아직 귀엽다고 본다 망씨랑 헬쥐중국건강 못따라가
2,2023-02-08,2.0,공매도 진짜 대단하네ㅋㅋㅋㅋㅋㅋㅋㅋㅋ ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,SK아이이테크놀로지,577.0,1.0,0.0,11.0,-1.0,공매도 진짜 대단하네
3,2023-02-03,2.0,띨엠전자우 자나깨나 암 조심,삼성전자우,373.0,5.0,12.0,34.0,1.0,띨엠전자우 자나깨나 암 조심
4,2023-02-02,2.0,휴 시외 5490에샀다가 물타서 겨우탈출했네 평단5310 5340매도... ...,코오롱모빌리티그룹,203.0,2.0,0.0,86.0,-1.0,휴 시외 에샀다가 물타서 겨우탈출했네 평단 매도 원에 추매 해서 관망좀할게요 무섭네...
...,...,...,...,...,...,...,...,...,...,...
14367,2023-02-07,1.0,악재는 소진되었고 이제부터 쭈욱 올라간다 지금잡아라 ~ 1차 115000원 물...,SKC,509.0,2.0,2.0,9.0,1.0,악재는 소진되었고 이제부터 쭈욱 올라간다 지금잡아라 차 원 물들어 올때 노저어라
14368,2023-02-07,3.0,고수로 인증받으려면? 최소 100억은 벌어야지??? ㅋㅋㅋ이 이후로부터는 돈이 돈...,삼성전자,265.0,0.0,0.0,370.0,1.0,고수로 인증받으려면 최소 억은 벌어야지 이 이후로부터는 돈이 돈을 벌게 하므로 그냥...
14369,2023-02-08,1.0,ai 비서?… 이랏샤이마세 여깁니다 형님들,웹케시,362.0,2.0,1.0,2.0,1.0,비서 이랏샤이마세 여깁니다 형님들
14370,2023-02-06,3.0,커피니? 맥심엄? 맥시멈을 맥심엄으로쓰는건 첨본다 푸하하하하하kida가 넘본다 했...,위메이드,373.0,0.0,0.0,81.0,1.0,커피니 맥심엄 맥시멈을 맥심엄으로쓰는건 첨본다 푸하하하하하 가 넘본다 했더니쫄았나 ...


In [54]:
d_test.reset_index(inplace=True)
d_test.drop('index',axis=1,inplace=True)
d_test

,날짜,label,제목+내용,기업명,조회수,공감수,비공감수,페이지,전날_대비_상승_하락,정제된 내용
0,2023-02-09,1.0,나 콜옵 6장 넣었다규 ㆍㆍㆍ? ㆍㆍ감사,삼성전자,220.0,0.0,0.0,197.0,-1.0,나 콜옵 장 넣었다규 감사
1,2023-02-09,2.0,중국풍선 40개국이상 스파이짓속보 삼성게시판은 잡벌레집합소됫노ㅋ,삼성전자,212.0,2.0,0.0,197.0,-1.0,중국풍선 개국이상 스파이짓속보 삼성게시판은 잡벌레집합소됫노
2,2023-02-09,3.0,조나단 에드워드에 의하면 거의 다ㅡ 지옥간다. 신앙은 철처히 양심과 그리스도로 인...,삼성전자,220.0,0.0,2.0,197.0,-1.0,조나단 에드워드에 의하면 거의 다 지옥간다 신앙은 철처히 양심과 그리스도로 인한 신...
3,2023-02-09,2.0,개 돼지의 나라 굿잡 https://www.youtube.com/watch?v=G...,삼성전자,225.0,2.0,0.0,199.0,-1.0,개 돼지의 나라 굿잡
4,2023-02-09,3.0,냉무 냉무냉무냉무냉무냉무냉무,삼성전자,218.0,0.0,0.0,199.0,-1.0,냉무 냉무냉무냉무냉무냉무냉무
...,...,...,...,...,...,...,...,...,...,...
6155,2023-02-02,2.0,1월 한달동안 공매도 20만주 갚았네 아직도 160만주 더갚아야되노ㅋㅋㅋㅋ,SK아이이테크놀로지,744.0,8.0,1.0,35.0,1.0,월 한달동안 공매도 만주 갚았네 아직도 만주 더갚아야되노
6156,2023-02-01,1.0,옥수수가 폭락한다고 글 싸자마자 반등하네 이새끼는 과학 그 자체다 진짜ㅋㅋㅋㅋ,SK아이이테크놀로지,500.0,11.0,2.0,35.0,1.0,옥수수가 폭락한다고 글 싸자마자 반등하네 이새끼는 과학 그 자체다 진짜
6157,2023-02-01,2.0,공모가 10만5천 언제갈래 대표 바뀐 기념으로 물린직원들을 위한 뭐 없냐,SK아이이테크놀로지,408.0,3.0,1.0,35.0,1.0,공모가 만 천 언제갈래 대표 바뀐 기념으로 물린직원들을 위한 뭐 없냐
6158,2023-02-01,3.0,인호 혼자서 이악물고 달려드는거 봐라 왜이리 화나있냐 캬캬캬캬,SK아이이테크놀로지,325.0,1.0,0.0,35.0,1.0,인호 혼자서 이악물고 달려드는거 봐라 왜이리 화나있냐 캬캬캬캬


In [75]:
d_test = d_test.drop("label", axis=1)

# 모델링
BERT는 이미 누군가가 학습해둔 모델을 사용한다(pre-trained model)는 것을 뜻한다. 따라서 사용하는 model과 tokenizer는 항상 mapping 관계여야 한다. 예를 들어서 U 팀이 개발한 BERT를 사용하는데, V팀이 개발한 BERT의 tokenizer를 사용하면 model은 텍스트를 이해할 수 없다. U팀의 BERT의 토크나이저는 '우리'라는 단어를 23번으로 int encoding하는 반면에, V라는 BERT의 tokenizer는 '우리'라는 단어를 103번으로 int encoding해 단어와 mapping 되는 정보 자체가 달라지기 때문이다. 이 부분은 뒤에서 간단히 진행해본 실습에서 더 자세히 다뤄볼 것이다.

## 1) 모델 학습

In [55]:
pip install gluonnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=689015 sha256=e803736e7d06606dcb0944df1a639d9442617394ce322b3b27d983c1036c92a4
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp


In [56]:
pip install mxnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [57]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [58]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-5jbryu3f/kobert-tokenizer_bc2d643beef743c0806131574aa21b18
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-5jbryu3f/kobert-tokenizer_bc2d643beef743c0806131574aa21b18
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4648 sha256=e133a8f1fc0d029ab4bec5129eb6fc5986b0e4e20683339490cbb40a53e375d6
  Stored in directory: /tmp/pip-ephem-wheel-cache-0cqjybu0/wheels/f7/cb/29/1a737fe71e5108dc30b04ea4a990f78ed271fa537aaf3fce7c
Successfully built kobert_tokenizer


In [59]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-65iuuraw
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-65iuuraw
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [60]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [61]:
#GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
# 윗부분 실행하면 에러남
# from kobert.pytorch_kobert import get_kobert_model
# from kobert_tokenizer import KoBERTTokenizer
# tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)

# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")
# model = AutoModel.from_pretrained("skt/kobert-base-v1")

In [70]:
# 1은 긍정, 2는 부정. 3이나 다른 값들은 삭제 필요
d_train["label"].value_counts()

3.0     5184
2.0     4639
1.0     4115
q          2
13.0       1
4.0        1
Name: label, dtype: int64

In [71]:
# 부정(2)을 0으로 바꿈 
d_train = d_train[d_train["label"].isin([1, 2])]
d_train.loc[d_train['label'] == 2.0, 'label'] = 0
d_train.loc[d_train['label'] == 1.0, 'label'] = 1
d_train["label"].value_counts()

0    4639
1    4115
Name: label, dtype: int64

In [ ]:
# # train_data에서 정제된 댓글, 라벨만 불러오기
# df_nn = train_data[["정제된 내용", "label"]] 

# # 기존 df_nn를 훈련 데이터와 테스트 데이터로 나눔
# from sklearn.model_selection import train_test_split
# dataset_train, dataset_test = train_test_split(df_nn,test_size=0.2,random_state=11)

In [72]:
# 댓글과 라벨만 사용, 훈련 데이터와 테스트 데이터 나눔
# 라벨링된 train 데이터만 사용
df_nn = d_train[["정제된 내용","label"]] 

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_nn["정제된 내용"],df_nn["label"],test_size=0.2,random_state=2023)

df_train = pd.concat([x_train,y_train],axis = 1)
df_test = pd.concat([x_test,y_test],axis = 1)

print(df_train["label"].unique())
df_train.head()

[1 0]


,정제된 내용,label
4603,옵티코어나 담아놔 내일 크게 먹어 상닫은거 군침흘리지말고,1
10391,끝인가요 잉자 다른물 찾아야 하나요,0
14036,개스피 박스피 괜히 나온말이 아니지,0
8061,빨간불로 종가 마무의리 스팸 계속 뜨면서 신용 못들어 오게 위로 간다 진정한 급등주...,1
7375,상투 정신 나간놈들만 이런 개잡주억 투자하지,0


In [ ]:
# dataset_train.shape, dataset_test.shape

In [73]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [105]:
# Setting parameters (일단 참고해서 임의로 지정)
max_len = 128
batch_size = 32 # cuda memory error. 64 -> 32 -> 16
warmup_ratio = 0.1
num_epochs = 5 # 조정해보자
max_grad_norm = 1
log_interval = 100
learning_rate =  0.00004

In [106]:
# 데이터셋을 koBERT 모델에 맞게 변형하는 클래스
# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding 등을 해주는 코드
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in dataset[sent_idx]]
        self.labels = [np.int32(i) for i in dataset[label_idx]]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  


# koBERT 모듈을 적재하여 모델을 학습시키는 분류기 
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [107]:
# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/NLP_PJT_3/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/NLP_PJT_3/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [108]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/NLP_PJT_3/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [109]:
# 훈련데이터, 테스트데이터 전처리
data_train = BERTDataset(df_train, "정제된 내용", "label", tok, max_len, True, False)
data_test = BERTDataset(df_test, "정제된 내용", "label", tok, max_len, True, False)

# 위에서 구현한 BERTDataset 클래스를 활용해 tokenization, int encoding, padding 을 진행
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [110]:
# 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [111]:
# optimizer, schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [112]:
# 정확도 측정을 위한 함수 정의
# 모델 학습

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

```
에러 발생
loss.backward()
RuntimeError: CUDA error: CUBLAS_STATUS_ALLOC_FAILED when calling `cublasCreate(handle)` 

-> 배치 사이즈 줄이는 해결책 (해결완료)
```

```
에러발생 (위 에러 해결하니까 갑자기 또 다른 에러)
token_ids = token_ids.long().to(device)
RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

-> 클래스 인덱스 조정의 문제.
Python은 0부터 읽으므로, 0, 1 로 조정해보자
```

In [113]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [114]:
train_history=[]
test_history=[]
loss_history=[]

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|          | 0/219 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7979786992073059 train acc 0.4375
epoch 1 batch id 101 loss 0.6621469855308533 train acc 0.5241336633663366
epoch 1 batch id 201 loss 0.5057005286216736 train acc 0.5833333333333334
epoch 1 train acc 0.5916043040757653


  0%|          | 0/55 [00:00<?, ?it/s]

epoch 1 test acc 0.706744071146245


  0%|          | 0/219 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7808300395256917


  0%|          | 0/219 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.04761746525764465 train acc 1.0
epoch 5 batch id 101 loss 0.22000576555728912 train acc 0.9464727722772277
epoch 5 batch id 201 loss 0.03678427264094353 train acc 0.9507151741293532
epoch 5 train acc 0.951457593438187


  0%|          | 0/55 [00:00<?, ?it/s]

epoch 5 test acc 0.7799160079051384


In [ ]:
# 모델 저장
torch.save(model, './KoBERT_model_dr_0.5.pt') 
torch.save(model.state_dict(), './model_state_dict_dr_0.5.pt')

---

## 2) 모델 예측
- 위에서 저장된 모델로, 게시글 내용에 따라 각각 neg, pos 라벨링 예측

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 설정
device = torch.device("cuda:0")

# 버트모델 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/NLP_PJT_3/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/NLP_PJT_3/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# Setting parameters (일단 참고해서 임의로 지정)
max_len = 128
batch_size = 16 # cuda memory error. 64 -> 32 -> 16
warmup_ratio = 0.1
num_epochs = 2 # 조정해보자
max_grad_norm = 1
log_interval = 100
learning_rate =  0.00004

In [ ]:
# KoBERT에 입력될 데이터셋 정리
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

In [ ]:
# 분류기 생성
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

---

In [ ]:
# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트

def getSentimentValue(comment, tok, max_len, batch_size, device):
  text_list = [] # 텍스트 데이터를 담을 리스트
  emo_list = [] # 감성 값을 담을 리스트
  for c in comment: # 모든 댓글
    text_list.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
  pdData = pd.DataFrame(text_list, columns = [['내용', '감성']] )
  pdData = pdData.values
  
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    # 이때, out이 예측 결과 리스트
    out = model(token_ids, valid_length, segment_ids)
	
    # e는 2가지 실수 값으로 구성된 리스트
    # 0번 인덱스가 더 크면 부정, 긍정은 반대
    for e in out:
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정
        value = 1
      emo_list.append(value)

  return emo_list # 텍스트 데이터에 1대1 매칭되는 감성값 리스트 반환


In [ ]:
test_label = getSentimentValue(d_test["정제된 내용"], tok, max_len, batch_size, device)
test_label

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,


In [ ]:
type(test_label)

list

In [ ]:
d_test["label"] = test_label
d_test

,날짜,기업명,제목+내용,조회수,공감수,페이지,전날_대비_상승_하락,정제된 내용,predicted_label,label
0,2023-02-09,삼성전자,누가 한민족 아니랄까봐 ㅋㅋㅋㅋㅋ 북한은 돼지딸 때매 난리고 남한은 개딸 때매 난리네,5,0,1,-1,누가 한민족 아니랄까봐 북한은 돼지딸 때매 난리고 남한은 개딸 때매 난리네,0,0
1,2023-02-09,삼성전자,2찍이들 몰살 해야해~~~~~~~~ 그래야 나라가 정상으로 간당,4,1,1,-1,찍이들 몰살 해야해 그래야 나라가 정상으로 간당,1,1
2,2023-02-09,삼성전자,[김일성주의1인투표]kbs뉴스도안보았나 김일성이나오고 김일성이가 수첩을드니 전부 수...,2,1,1,-1,김일성주의 인투표 뉴스도안보았나 김일성이나오고 김일성이가 수첩을드니 전부 수첩을 한...,0,0
3,2023-02-09,삼성전자,존귀하신 김주애 ~~~~ 존귀하신 문다혜~~~~ 누구 딸이더라???,10,0,1,-1,존귀하신 김주애 존귀하신 문다혜 누구 딸이더라,1,1
4,2023-02-09,삼성전자,굿짐똥당 대표경선좀바라 푸헤헤헤헤 푸헤헤헤헤 푸헤헤헤헤 푸헤헤헤헤 푸헤헤헤헤 푸헤...,11,2,1,-1,굿짐똥당 대표경선좀바라 푸헤헤헤헤 푸헤헤헤헤 푸헤헤헤헤 푸헤헤헤헤 푸헤헤헤헤 푸헤헤헤헤,0,0
...,...,...,...,...,...,...,...,...,...,...
10722,2022-12-28,SK아이이테크놀로지,걸레 맞네요 강제 장투로 전환ㅠㅠ,404,4,60,-1,걸레 맞네요 강제 장투로 전환,,0
10723,2022-12-28,SK아이이테크놀로지,"45,000원~65,000원 앞으로 3년~~박스권이다",519,2,60,-1,원 원 앞으로 년 박스권이다,,0
10724,2022-12-28,SK아이이테크놀로지,옥수수명언 中 차트 분석에는 봉에 대한 분석과 각도에 대한 분석이 있소. 옥수수는...,670,8,60,-1,옥수수명언 차트 분석에는 봉에 대한 분석과 각도에 대한 분석이 있소 옥수수는 봉과 ...,,0
10725,2022-12-28,SK아이이테크놀로지,얜 배당도 안주면서 개폭이네 ㅋ ㅋ,312,1,60,-1,얜 배당도 안주면서 개폭이네,,0


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append(0)
            elif np.argmax(logits) == 1:
                test_eval.append(1)
    
    return test_eval

In [ ]:
# # 예측 모델 설정
# def predict(predict_sentence):

#     data = [predict_sentence, '0']
#     dataset_another = [data]

#     another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
#     test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
#     model.eval()

#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)

#         valid_length= valid_length
#         label = label.long().to(device)

#         out = model(token_ids, valid_length, segment_ids)

#         test_eval=[]
#         for i in out:
#             logits=i
#             logits = logits.detach().cpu().numpy()
            
#             min_v = min(logits)
#             total = 0
#             probability = []
#             logits = np.round(new_softmax(logits), 3).tolist()
#             for logit in logits:
#                 print(logit)
#                 probability.append(np.round(logit, 3))

#             if np.argmax(logits) == 0: emotion = "fear"
#             elif np.argmax(logits) == 1: emotion = 'greed'

#             probability.append(emotion)
#             print(probability)
#     return probability

In [ ]:
# 학습 모델 로드
model = torch.load('./KoBERT_model_dr_0.5.pt')
model.load_state_dict(torch.load('./model_state_dict_dr_0.5.pt'))

<All keys matched successfully>

In [ ]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/NLP_PJT_3/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


---